In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
# clean data
data = pd.read_csv('original_data.csv', index_col=0)
data = data.drop('AppointmentID', axis=1)
data.rename(columns = {'Hipertension':'Hypertension','Handcap':'Handicap','No-show':'Noshow'}, inplace=True)
data.Gender = [1 if i=='M' else 0 for i in data.Gender]
data.Noshow = [1 if i=='Yes' else 0 for i in data.Noshow]
data.ScheduledDay = data.ScheduledDay.apply(np.datetime64)
data.AppointmentDay = data.AppointmentDay.apply(np.datetime64)
data = data[data.Age>=0]
#data.to_csv('cleaned_data.csv')
print(data.shape)
data.head()

(110526, 12)


,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,Noshow
PatientId,,,,,,,,,,,,
2.987250e+13,0,2016-04-29 18:38:08,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,0
5.589978e+14,1,2016-04-29 16:08:27,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,0
4.262962e+12,0,2016-04-29 16:19:04,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,0
8.679512e+11,0,2016-04-29 17:29:31,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0
8.841186e+12,0,2016-04-29 16:07:23,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,0


In [26]:
print ("{}% of the patients were no-shows.".format(round(100*sum(data.Noshow)/data.shape[0],2)))

20.19% of the patients were no-shows.


In [34]:
(data.AppointmentDay - data.ScheduledDay)

PatientId
2.987250e+13   -1 days +05:21:52
5.589978e+14   -1 days +07:51:33
4.262962e+12   -1 days +07:40:56
8.679512e+11   -1 days +06:30:29
8.841186e+12   -1 days +07:52:37
9.598513e+13     1 days 15:23:09
7.336882e+14     1 days 08:54:48
3.449833e+12     1 days 08:20:02
5.639473e+13   -1 days +15:57:44
7.812456e+13     1 days 11:11:35
7.345362e+14     1 days 09:01:49
7.542951e+12     2 days 15:15:48
5.666548e+14     0 days 12:26:09
9.113946e+14     0 days 09:07:53
9.988472e+13     0 days 13:53:36
9.994839e+10     2 days 15:12:33
8.457439e+13     0 days 15:08:13
1.479497e+13     0 days 14:31:03
1.713538e+13     2 days 13:05:42
7.223289e+12   -1 days +13:16:46
6.222575e+14     1 days 16:08:46
1.215484e+13     1 days 13:09:15
8.632298e+14     3 days 10:30:44
2.137540e+14     0 days 13:32:55
8.734858e+12   -1 days +09:40:41
5.819370e+12     2 days 08:55:43
2.578785e+10   -1 days +09:40:18
1.215484e+13     1 days 13:08:15
5.926172e+12   -1 days +08:11:58
1.225776e+12   -1 days +08:43:31
